# Clustering - step 1

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [5]:
#install Beautiful Soup / requests
!pip install BeautifulSoup4
!pip install requests

In [6]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [7]:
raw = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(raw.text, 'lxml')

In [47]:
#get data from the page
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #set header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert to df
c_df = pd.DataFrame(data = data,columns = columns)

c_df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [48]:
#delete not assign
c_df = c_df[c_df['Borough'] != 'Not assigned']
c_df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [50]:
c_df.shape

(103, 3)

# step 2

In [57]:
ll = pd.read_csv('https://cocl.us/Geospatial_data')
ll.columns =['PC', 'Latitude','Longitude'] 
ll.head()

,PC,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [63]:
final=c_df.join(ll.set_index('PC'), on='Postal code')
final.columns=['Postalcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
final.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
5,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


# step 3

In [68]:
!pip install folium
import folium # to create a map of toronto

     |████████████████████████████████| 92kB 8.6MB/s eta 0:00:011


In [74]:
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_t = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10)
map_t

In [77]:
for lat, lng, borough, neighborhood in zip(final['Latitude'], final['Longitude'], final['Borough'], final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=1).add_to(map_t)  
map_t

In [78]:
#foursquare
CLIENT_ID = 'V5W4CDD1NY1CY0KIL22NCAHNKF2CZRI1YRR2FDX3ZX5XE0L1' 
CLIENT_SECRET = 'DWNADAJR4OBB05ZQLIR4QZ5CDGSAUEGOWDNJPXAVEYLMHTR3' 
VERSION = '20200414' # Foursquare API version

In [108]:
downtown = final[final['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,Richmond / Adelaide / King,43.650571,-79.384568
8,M5J,Downtown Toronto,Harbourfront East / Union Station / Toronto Is...,43.640816,-79.381752
9,M5K,Downtown Toronto,Toronto Dominion Centre / Design Exchange,43.647177,-79.381576


In [83]:
# get top 5 venues around St. James Town in Downtown Toronto
neighborhood_latitude = downtown.loc[0, 'Latitude'] 
neighborhood_longitude = downtown.loc[0, 'Longitude'] 

neighborhood_name = downtown.loc[0, 'Neighborhood']

LIMIT = 5
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, 43.651494, -79.375418, VERSION, radius, LIMIT)

venues = requests.get(url).json()
venues

{'meta': {'code': 200, 'requestId': '5e95c6630de0d9001b737e6d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'St. Lawrence',
  'headerFullLocation': 'St. Lawrence, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 85,
  'suggestedBounds': {'ne': {'lat': 43.6559940045, 'lng': -79.36921028605637},
   'sw': {'lat': 43.6469939955, 'lng': -79.38162571394362}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '574ad72238fa943556d93b8e',
       'name': 'Gyu-Kaku Japanese BBQ',
       'location': {'address': '81 Church St',
        'crossStreet': 'at Adelaide St E',
        'lat': 43.651422275497914,
        'lng': -79.37504693687086,
        'labeledLatLngs': [{'

In [84]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [87]:
import json # library of JSON file
from pandas.io.json import json_normalize # tranform JSON file

venues1 = venues['response']['groups'][0]['items']  
nearbyvenues = json_normalize(venues1) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearbyvenues =nearbyvenues.loc[:, filtered_columns]

# clean category
nearbyvenues['venue.categories'] = nearbyvenues.apply(get_category_type, axis=1)

# clean columns
nearbyvenues.columns = [col.split(".")[-1] for col in nearbyvenues.columns]

nearbyvenues.head()

,name,categories,lat,lng
0,Gyu-Kaku Japanese BBQ,Japanese Restaurant,43.651422,-79.375047
1,GoodLife Fitness Toronto 137 Yonge Street,Gym,43.651242,-79.378068
2,GEORGE Restaurant,Restaurant,43.653346,-79.374445
3,Fahrenheit Coffee,Coffee Shop,43.652384,-79.372719
4,Aveda Institute Toronto,Cosmetics Shop,43.650096,-79.373630


In [89]:
#get neighborhood names in downtown toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

downtown_venues = getNearbyVenues(names=downtown['Neighborhood'],
                                   latitudes=downtown['Latitude'],
                                   longitudes=downtown['Longitude'])
downtown_venues.head()

Regent Park / Harbourfront
Queen's Park / Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst
 Quay / South Niagara / Island airport
Rosedale
Stn A PO Boxes
St. James Town / Cabbagetown
First Canadian Place / Underground city
Church and Wellesley


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park / Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park / Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park / Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park / Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [90]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,5,5,5,5,5,5
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst\n Quay / South Niagara / Island airport,5,5,5,5,5,5
Central Bay Street,5,5,5,5,5,5
Christie,5,5,5,5,5,5
Church and Wellesley,5,5,5,5,5,5
Commerce Court / Victoria Hotel,5,5,5,5,5,5
First Canadian Place / Underground city,5,5,5,5,5,5
"Garden District, Ryerson",5,5,5,5,5,5
Harbourfront East / Union Station / Toronto Islands,5,5,5,5,5,5


In [91]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 47 uniques categories.


In [93]:
# onehot --> oh
dt_oh = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column
dt_oh['Neighborhood'] = downtown_venues['Neighborhood'] 

# neighborhood column --> 1st column
fixed_columns = [dt_oh.columns[-1]] + list(dt_oh.columns[:-1])
dt_oh = dt_oh[fixed_columns]

dt_oh.head()

,Vegetarian / Vegan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Bakery,Breakfast Spot,Bubble Tea Shop,...,Pub,Ramen Restaurant,Restaurant,Salad Place,Spa,Sporting Goods Shop,Tea Room,Thai Restaurant,Theme Restaurant,Trail
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
dt_grouped = dt_oh.groupby('Neighborhood').mean().reset_index()
dt_grouped.head()

,Neighborhood,Vegetarian / Vegan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Bakery,Breakfast Spot,...,Pub,Ramen Restaurant,Restaurant,Salad Place,Spa,Sporting Goods Shop,Tea Room,Thai Restaurant,Theme Restaurant,Trail
0,Berczy Park,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CN Tower / King and Spadina / Railway Lands / ...,0.0,0.2,0.2,0.2,0.2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Church and Wellesley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,...,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0


In [99]:
#get top 3 venues per neighborhood
num_top_venues = 3
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

# create top venues column
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
nb_venues = pd.DataFrame(columns=columns)
nb_venues['Neighborhood'] = dt_grouped['Neighborhood']

for ind in np.arange(dt_grouped.shape[0]):
    nb_venues.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

nb_venues

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Berczy Park,Vegetarian / Vegan Restaurant,Restaurant,Liquor Store
1,CN Tower / King and Spadina / Railway Lands / ...,Harbor / Marina,Airport,Airport Food Court
2,Central Bay Street,Coffee Shop,Gastropub,Middle Eastern Restaurant
3,Christie,Café,Italian Restaurant,Coffee Shop
4,Church and Wellesley,Theme Restaurant,Dance Studio,Ramen Restaurant
5,Commerce Court / Victoria Hotel,Gym,Café,Restaurant
6,First Canadian Place / Underground city,Restaurant,Gym,American Restaurant
7,"Garden District, Ryerson",Café,Tea Room,Comic Shop
8,Harbourfront East / Union Station / Toronto Is...,Park,Dessert Shop,Sporting Goods Shop
9,Kensington Market / Chinatown / Grange Park,Arts & Crafts Store,Bakery,Mexican Restaurant


In [109]:
#K-means clustering into 5 groups

# import k-means
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

dt_grouped_clustering = dt_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 1, 4, 2, 3, 3, 4, 2, 4], dtype=int32)

In [116]:
downtown.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [124]:
nb_venues.insert(1,'Cluster', kmeans.labels_)

In [125]:
nb_venues.head()

,Cluster Label,Cluster,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,0,0,0,Berczy Park,Vegetarian / Vegan Restaurant,Restaurant,Liquor Store
1,2,2,2,CN Tower / King and Spadina / Railway Lands / ...,Harbor / Marina,Airport,Airport Food Court
2,1,1,1,Central Bay Street,Coffee Shop,Gastropub,Middle Eastern Restaurant
3,4,4,4,Christie,Café,Italian Restaurant,Coffee Shop
4,2,2,2,Church and Wellesley,Theme Restaurant,Dance Studio,Ramen Restaurant


In [127]:
nb_venues = nb_venues.drop(columns=['Cluster Label', 'Cluster Labels'])
nb_venues.head()

,Cluster,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,0,Berczy Park,Vegetarian / Vegan Restaurant,Restaurant,Liquor Store
1,2,CN Tower / King and Spadina / Railway Lands / ...,Harbor / Marina,Airport,Airport Food Court
2,1,Central Bay Street,Coffee Shop,Gastropub,Middle Eastern Restaurant
3,4,Christie,Café,Italian Restaurant,Coffee Shop
4,2,Church and Wellesley,Theme Restaurant,Dance Studio,Ramen Restaurant


In [128]:
# add latitude/longitude for each neighborhood
dt_merged = downtown.join(nb_venues.set_index('Neighborhood'), on='Neighborhood')

dt_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,1,Distribution Center,Spa,Bakery
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,1,Coffee Shop,Park,Distribution Center
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Café,Tea Room,Comic Shop
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Gym,Japanese Restaurant,Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Vegetarian / Vegan Restaurant,Restaurant,Liquor Store


In [130]:
#examine cluster 0
dt_merged.loc[dt_merged['Cluster'] == 0, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
4,Downtown Toronto,0,Vegetarian / Vegan Restaurant,Restaurant,Liquor Store
7,Downtown Toronto,0,Vegetarian / Vegan Restaurant,Hotel,Restaurant
15,Downtown Toronto,0,Vegetarian / Vegan Restaurant,Restaurant,Museum


In [131]:
#examine cluster 1
dt_merged.loc[dt_merged['Cluster'] == 1, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Downtown Toronto,1,Distribution Center,Spa,Bakery
1,Downtown Toronto,1,Coffee Shop,Park,Distribution Center
5,Downtown Toronto,1,Coffee Shop,Gastropub,Middle Eastern Restaurant


In [132]:
#examine cluster 2
dt_merged.loc[dt_merged['Cluster'] == 2, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
8,Downtown Toronto,2,Park,Dessert Shop,Sporting Goods Shop
13,Downtown Toronto,2,Harbor / Marina,Airport,Airport Food Court
14,Downtown Toronto,2,Park,Playground,Trail
18,Downtown Toronto,2,Theme Restaurant,Dance Studio,Ramen Restaurant


In [133]:
#examine cluster 3
dt_merged.loc[dt_merged['Cluster'] == 3, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
3,Downtown Toronto,3,Gym,Japanese Restaurant,Restaurant
9,Downtown Toronto,3,Gym,Hotel,Restaurant
10,Downtown Toronto,3,Gym,Café,Restaurant
11,Downtown Toronto,3,Dessert Shop,Italian Restaurant,Japanese Restaurant
17,Downtown Toronto,3,Restaurant,Gym,American Restaurant


In [134]:
#examine cluster 4
dt_merged.loc[dt_merged['Cluster'] == 4, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
2,Downtown Toronto,4,Café,Tea Room,Comic Shop
6,Downtown Toronto,4,Café,Italian Restaurant,Coffee Shop
12,Downtown Toronto,4,Arts & Crafts Store,Bakery,Mexican Restaurant
16,Downtown Toronto,4,Diner,Indian Restaurant,Italian Restaurant


In [139]:
#visualize on map

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
dtmap_clusters = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start=1)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['Latitude'], dt_merged['Longitude'], dt_merged['Neighborhood'], dt_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(dtmap_clusters)
dtmap_clusters